# Problem 1
## 1. 
The firm's problem is 
$$
max_{q} E[(a-bq)q - cq],
$$
where 
$$
b \sim logN(\mu, \sigma).
$$
Let $g(q) = E[(a-bq)q - cq]$, the expected profit for given quanity. Note that this function is linear in $b$. Thus
$$
g(q) = E[(a-bq)q - cq] = (a - E[b]q)q - cq = (a - e^{\mu + \frac{\sigma^2}{2}}q)q - cq.
$$
So the question of the firm is now
$$
max_{q} {(a - e^{\mu + \frac{\sigma^2}{2}}q)q - cq}.
$$
The FOC with respect to quantity is 
$$
a - 2bq - c = 0,
$$
which gives solution
$$
q^* = \frac{c - a}{2e^{\mu + \frac{\sigma^2}{2}}}.
$$
Note that the firm cannot supply a negative quantity, so if $a \leq c$, the firm simply stops production.

## 2.

In [2]:
using InstantiateFromURL
activate_github("QuantEcon/QuantEconLectureAllPackages", tag = "v0.9.5");
using LinearAlgebra, Statistics, Compat
using QuantEcon, QuadGK, FastGaussQuadrature, Distributions, Expectations
using Interpolations, Plots, LaTeXStrings, ProgressMeter
using Compat, Random
# this might take some time
function profit_max_q(a,c,mu,sigma,method,n)  
    if a <= c
        println("Not a valid profit function!");
        return 0;
    end
    if (method!="mc")&(method!="quad")
        println("Not a valid option!");
    end
    if method == "mc"
        return monte_carlo(a,c,mu,sigma,n);
    elseif method == "quad"
        return quadrature(a,c,mu,sigma,n);
    end
end

function monte_carlo(a,c,mu,sigma,n)
    prob = rand(n);      #random draws of probability                       
    b_hat = quantile.(LogNormal(mu, sigma), prob);     #used to hold estimation of b
    quantity = collect(0:10e-6:(a/(minimum(b_hat))));    #generating a grid of quantities
    profit = (a .- mean(b_hat)quantity).*quantity - c*quantity      #calculate profit
    info = findmax(profit);      #record best strategy of the firm
    return quantity[info[2]];      #return best strategy of the firm
end

function quadrature(a,c,mu,sigma,n)
    quantity = collect(0:10e-6:exp(mu*mu + sigma*sigma));    #generating a grid of quantities; need larger lower bound 
    times = length(quantity);      #get the length of the vector
    dist = LogNormal(mu, sigma);      #density of b
    kernel = expectation(dist);
    index = 1;       #initialize loop control
    profit = zeros(length(quantity));      #initialize profit vector
    node, weight = qnwlege(n, 0, 100);      #nodes and weights used in integration
    
    while index <= times      #this loop computes the expected profit for given quantities
        g(b) = pdf.(LogNormal(mu,sigma), b)*((a - b*quantity[index])*quantity[index] - c*quantity[index]);
        profit[index] = weight ⋅ g.(node);
        index = index + 1;
    end
    
    info = findmax(profit);      #record best strategy of the firm
    return quantity[info[2]];      #return best strategy of the firm
end


┌ Info: Recompiling stale cache file /Users/btan99/.julia/compiled/v1.0/InstantiateFromURL/vAXbt.ji for InstantiateFromURL [43edad99-fa64-5e4f-9937-1c09a410b73f]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/btan99/.julia/compiled/v1.0/QuantEcon/V0Mv9.ji for QuantEcon [fcd29c91-0bd7-5a09-975d-7ac3f643a60c]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/btan99/.julia/compiled/v1.0/Expectations/r1e2K.ji for Expectations [2fe49d83-0758-5602-8f54-1f90ad0d522b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/btan99/.julia/compiled/v1.0/Interpolations/VpKVx.ji for Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/btan99/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /Users/btan99/.julia/compiled/v1.0/ProgressMeter/3V8n6.ji for ProgressMeter [92933f4c-e287-5

quadrature (generic function with 1 method)

## 3. 

In [4]:
#initial values
a = 2;
c = 1; 
mu = 1;
sigma = 1;
n = 10;

profit_max_q(a,c,mu,sigma,"quad",n)      #numerical integration is slow
profit_max_q(a,c,mu,sigma,"mc",n)

#another set of values
a = 4;
c = 2; 
mu = 1.5;
sigma = 0.9;
n = 50;

profit_max_q(a,c,mu,sigma,"quad",n)      #numerical integration is slow
profit_max_q(a,c,mu,sigma,"mc",n)



0.12171000000000001

# Problem 2

In [63]:
n = 100000;
x = rand(n);      #random vector in the first coordinate
y = rand(n);      #random vector in the second coordinate
index = 1;      #initialize index
count = 0;      #initialize the number points that fall into the unit circle
while index <= n
    if (x[index] - 0.5)^2 + (y[index] - 0.5)^2 <= 0.25
        count += 1;
    end
    index += 1;
end

pi_hat = (count/n)*4      #estimate pi

3.14568

# Problem 3
## N/A

## Problem 4
# 1.

In [78]:
v_0 = rand(20000,20000);      #generate a 20000 * 20000 random vector

## 2.

In [74]:
function exp_cols(v)
    index = 1;      #initialize index
    while index <= 20000
        v[:, index] = exp.(v[:, index])
        index += 1;
    end
    return v;
end

exp_cols (generic function with 1 method)

## 3.

In [80]:
function exp_rows(v)
    index = 1;      #initialize index
    while index <= 20000
        v[index, :] = exp.(v[index, :]);
        index += 1;
    end
    return v;
end

exp_rows (generic function with 1 method)

## 4. 

In [103]:
exp_cols(v_0);
t_1 = @elapsed exp_cols(v_0);

exp_rows(v_0);
t_2 = @elapsed exp_rows(v_0);

println("The difference of running time column vs row is ",  t_2 - t_1 )      #calculate the difference

The difference of running time column vs row is 16.895963664


## 5. 

We see that exponentiation by rows is more costly than by columnns.